In [ ]:
import torch
import torch.nn as nn
import instruction_test_data as itd
import numpy as np
import card_embedding as ce
import nesting
from multi_head_attention import MultiHeadAttention
from positional_embedding import PositionalEmbedding

d=8
shared_embedding_holder = ce.SharedEmbeddingHolder(d, device='cuda')
positional_embedding = PositionalEmbedding(d, device='cuda')

In [3]:
instruction_embedding = ce.InstructionEmbedding(shared_embedding_holder, d, device="cuda")

instruction_embeddings = instruction_embedding(itd.instructions_batch)
print(instruction_embeddings)
print(instruction_embeddings.size())

tensor([[-4.2341,  7.5805,  2.9517, 10.1517,  3.3741,  0.8405, -0.0766,  9.8363],
        [ 4.7402,  8.0390, -1.6629, -1.0041,  0.9761,  7.3777,  5.7191, 11.6826],
        [ 2.2836, -0.3849, -1.4053,  1.6018, -0.5692,  2.5320, -1.0775, -0.6356],
        [ 2.2868, -0.4213, -1.4796,  1.6179, -0.6306,  2.6436, -1.2053, -0.6050],
        [ 7.3646,  2.1420,  4.3850, -0.6955,  2.6332,  8.7007, -5.1941,  6.3823]],
       device='cuda:0', grad_fn=<SumBackwardAutogradNestedTensor1>)
torch.Size([5, 8])
